In [1]:
pip install requests beautifulsoup4

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import requests
from bs4 import BeautifulSoup
import os
import re
import urllib3

In [4]:
# Desabilitar avisos relacionados a requisições inseguras, ou seja, quando o certificado SSL do servidor não é verificado corretamente
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [13]:
# URL base do site
base_url = "https://acordaos.economia.gov.br"

# Iniciar uma sessão de requests
session = requests.Session()

# Desabilitar avisos de certificado SSL
requests.packages.urllib3.disable_warnings()

# Função para criar pasta se não existir
def criar_pasta(pasta):
    if not os.path.exists(pasta):
        os.makedirs(pasta)

# Função para baixar o PDF
def download_pdf(pdf_url, caminho_pdf):
    response = session.get(pdf_url, verify=False)
    if response.status_code == 200:
        # Salvar o conteúdo do PDF no caminho especificado
        with open(caminho_pdf, 'wb') as f:
            f.write(response.content)
        print(f"PDF baixado: {caminho_pdf}")
    else:
        print(f"Falha ao baixar o PDF: {pdf_url}")

# Função para fazer o scraping de uma página e retornar links de PDF e próximo link
def scrape_page(url):
    response = session.get(url, verify=False)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar todos os links de PDFs na página
    pdf_links = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.endswith('.pdf'):
            if href.startswith('http'):
                pdf_url = href
            else:
                pdf_url = base_url + href
            pdf_links.append(pdf_url)

    # Procurar o link para a próxima página (usando a classe 'next-page')
    next_page = soup.find('a', class_='next-page', href=True)
    if next_page:
        next_url = base_url + next_page['href']  # Concatenar com o base_url
        return pdf_links, next_url
    else:
        return pdf_links, None

# Variável base_path (onde os arquivos serão armazenados)
base_path = "/content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos"


In [14]:
links_por_grupo = {
    "1 OBRIGATORIEDADE DE APRESENTAÇÃO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+OBRIGATORIEDADE+DE+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+OBRIGATORIEDADE+DE+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+OBRIGATORIEDADE+DE+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+OBRIGATORIEDADE+DE+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+OBRIGATORIEDADE+DE+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s%3A%222019%22"
    ],
    "2 DESCONTO SIMPLIFICADO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DESCONTO+SIMPLIFICADO&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DESCONTO+SIMPLIFICADO&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DESCONTO+SIMPLIFICADO&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DESCONTO+SIMPLIFICADO&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DESCONTO+SIMPLIFICADO&fq=ano_publicacao_s%3A%222019%22"
    ],
}


In [15]:
links_por_grupo.update({
    "3 PRAZO PARA APRESENTAÇÃO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+PRAZO+PARA+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s:%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+PRAZO+PARA+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s:%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+PRAZO+PARA+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s:%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+PRAZO+PARA+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s:%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+PRAZO+PARA+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s:%222019%22"
    ],
    "4 MULTA POR ATRASO NA ENTREGA DA DECLARAÇÃO OU NÃO APRESENTAÇÃO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+MULTA+POR+ATRASO+NA+ENTREGA+DA+DECLARA%C3%87%C3%83O+OU+N%C3%83O+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s:%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+MULTA+POR+ATRASO+NA+ENTREGA+DA+DECLARA%C3%87%C3%83O+OU+N%C3%83O+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s:%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+MULTA+POR+ATRASO+NA+ENTREGA+DA+DECLARA%C3%87%C3%83O+OU+N%C3%83O+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s:%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+MULTA+POR+ATRASO+NA+ENTREGA+DA+DECLARA%C3%87%C3%83O+OU+N%C3%83O+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s:%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+MULTA+POR+ATRASO+NA+ENTREGA+DA+DECLARA%C3%87%C3%83O+OU+N%C3%83O+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s:%222019%22"
    ],
    "5 PREENCHIMENTO IRPF 2024": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=PREENCHIMENTO+IRPF&fq=ano_publicacao_s:%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=PREENCHIMENTO+IRPF&fq=ano_publicacao_s:%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=PREENCHIMENTO+IRPF&fq=ano_publicacao_s:%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=PREENCHIMENTO+IRPF&fq=ano_publicacao_s:%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=PREENCHIMENTO+IRPF&fq=ano_publicacao_s:%222019%22"
    ],
    "6 TRANSMISSAO DA DECLARAÇÃO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+TRANSMISSAO+DA+DECLARA%C3%87%C3%83O&fq=ano_publicacao_s:%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+TRANSMISSAO+DA+DECLARA%C3%87%C3%83O&fq=ano_publicacao_s:%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+TRANSMISSAO+DA+DECLARA%C3%87%C3%83O&fq=ano_publicacao_s:%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+TRANSMISSAO+DA+DECLARA%C3%87%C3%83O&fq=ano_publicacao_s:%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+TRANSMISSAO+DA+DECLARA%C3%87%C3%83O&fq=ano_publicacao_s:%222019%22"
    ],
    "7 LOCAL DE APRESENTAÇÃO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+LOCAL+DE+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s:%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+LOCAL+DE+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s:%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+LOCAL+DE+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s:%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+LOCAL+DE+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s:%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+LOCAL+DE+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s:%222019%22"
    ],
    "8 DECLARAÇÃO DE EXERCÍCIOS ANTERIORES": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+DECLARA%C3%87%C3%83O+DE+EXERC%C3%8DCIOS+ANTERIORES&fq=ano_publicacao_s:%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+DECLARA%C3%87%C3%83O+DE+EXERC%C3%8DCIOS+ANTERIORES&fq=ano_publicacao_s:%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+DECLARA%C3%87%C3%83O+DE+EXERC%C3%8DCIOS+ANTERIORES&fq=ano_publicacao_s:%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+DECLARA%C3%87%C3%83O+DE+EXERC%C3%8DCIOS+ANTERIORES&fq=ano_publicacao_s:%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+DECLARA%C3%87%C3%83O+DE+EXERC%C3%8DCIOS+ANTERIORES&fq=ano_publicacao_s:%222019%22"
    ],
    "9 RETIFICAÇÃO DA DECLARAÇÃO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+RETIFICA%C3%87%C3%83O+DA+DECLARA%C3%87%C3%83O&fq=ano_publicacao_s:%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+RETIFICA%C3%87%C3%83O+DA+DECLARA%C3%87%C3%83O&fq=ano_publicacao_s:%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+RETIFICA%C3%87%C3%83O+DA+DECLARA%C3%87%C3%83O&fq=ano_publicacao_s:%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+RETIFICA%C3%87%C3%83O+DA+DECLARA%C3%87%C3%83O&fq=ano_publicacao_s:%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+RETIFICA%C3%87%C3%83O+DA+DECLARA%C3%87%C3%83O&fq=ano_publicacao_s:%222019%22"
    ],
    "10 COMPROVANTE DE RENDIMENTOS": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+COMPROVANTE+DE+RENDIMENTOS+&fq=ano_publicacao_s:%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+COMPROVANTE+DE+RENDIMENTOS+&fq=ano_publicacao_s:%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+COMPROVANTE+DE+RENDIMENTOS+&fq=ano_publicacao_s:%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+COMPROVANTE+DE+RENDIMENTOS+&fq=ano_publicacao_s:%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+COMPROVANTE+DE+RENDIMENTOS+&fq=ano_publicacao_s:%222019%22"
    ],
    "11 CADASTRO DE PESSOAS FÍSICAS": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+CADASTRO+DE+PESSOAS+F%C3%8DSICAS&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+CADASTRO+DE+PESSOAS+F%C3%8DSICAS&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+CADASTRO+DE+PESSOAS+F%C3%8DSICAS&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+CADASTRO+DE+PESSOAS+F%C3%8DSICAS&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+CADASTRO+DE+PESSOAS+F%C3%8DSICAS&fq=ano_publicacao_s%3A%222019%22"
    ],
    "12 CÁLCULO E PAGAMENTO DO IMPOSTO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+C%C3%81LCULO+E+PAGAMENTO+DO+IMPOSTO&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+C%C3%81LCULO+E+PAGAMENTO+DO+IMPOSTO&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+C%C3%81LCULO+E+PAGAMENTO+DO+IMPOSTO&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+C%C3%81LCULO+E+PAGAMENTO+DO+IMPOSTO&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+C%C3%81LCULO+E+PAGAMENTO+DO+IMPOSTO&fq=ano_publicacao_s%3A%222019%22"
    ],
    "13 RESTITUIÇÃO/COMPENSAÇÃO DO IR": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RESTITUI%C3%87%C3%83O+e+COMPENSA%C3%87%C3%83O&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RESTITUI%C3%87%C3%83O+e+COMPENSA%C3%87%C3%83O&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RESTITUI%C3%87%C3%83O+e+COMPENSA%C3%87%C3%83O&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RESTITUI%C3%87%C3%83O+e+COMPENSA%C3%87%C3%83O&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RESTITUI%C3%87%C3%83O+e+COMPENSA%C3%87%C3%83O&fq=ano_publicacao_s%3A%222019%22"
    ],
    "15 DECLARAÇÃO EM CONJUNTO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DECLARA%C3%87%C3%83O+EM+CONJUNTO+&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DECLARA%C3%87%C3%83O+EM+CONJUNTO+&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DECLARA%C3%87%C3%83O+EM+CONJUNTO+&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DECLARA%C3%87%C3%83O+EM+CONJUNTO+&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DECLARA%C3%87%C3%83O+EM+CONJUNTO+&fq=ano_publicacao_s%3A%222019%22"
    ],
    "16 DECLARAÇÃO EM SEPARADO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DECLARA%C3%87%C3%83O+EM+SEPARADO&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DECLARA%C3%87%C3%83O+EM+SEPARADO&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DECLARA%C3%87%C3%83O+EM+SEPARADO&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DECLARA%C3%87%C3%83O+EM+SEPARADO&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DECLARA%C3%87%C3%83O+EM+SEPARADO&fq=ano_publicacao_s%3A%222019%22"
    ],
    "17 ESPÓLIO - CONTRIBUINTE FALECIDO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ESP%C3%93LIO+CONTRIBUINTE+FALECIDO+&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ESP%C3%93LIO+CONTRIBUINTE+FALECIDO+&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ESP%C3%93LIO+CONTRIBUINTE+FALECIDO+&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ESP%C3%93LIO+CONTRIBUINTE+FALECIDO+&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ESP%C3%93LIO+CONTRIBUINTE+FALECIDO+&fq=ano_publicacao_s%3A%222019%22"
    ],
    "18 EXTERIOR": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+EXTERIOR&fq=ano_publicacao_s:%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+EXTERIOR&fq=ano_publicacao_s:%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+EXTERIOR&fq=ano_publicacao_s:%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+EXTERIOR&fq=ano_publicacao_s:%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=IRPF+e+EXTERIOR&fq=ano_publicacao_s:%222019%22"
    ],
    "19 RENDIMENTOS TRIBUTÁVEIS - TRABALHO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+TRIBUT%C3%81VEIS+e+TRABALHO+&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+TRIBUT%C3%81VEIS+e+TRABALHO+&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+TRIBUT%C3%81VEIS+e+TRABALHO+&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+TRIBUT%C3%81VEIS+e+TRABALHO+&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+TRIBUT%C3%81VEIS+e+TRABALHO+&fq=ano_publicacao_s%3A%222019%22"
    ],
    "20 RENDIMENTOS TRIBUTÁVEIS – ALUGUÉIS": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=RENDIMENTOS+TRIBUT%C3%81VEIS+e+ALUGU%C3%89IS&fq=ano_publicacao_s:%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=RENDIMENTOS+TRIBUT%C3%81VEIS+e+ALUGU%C3%89IS&fq=ano_publicacao_s:%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=RENDIMENTOS+TRIBUT%C3%81VEIS+e+ALUGU%C3%89IS&fq=ano_publicacao_s:%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=RENDIMENTOS+TRIBUT%C3%81VEIS+e+ALUGU%C3%89IS&fq=ano_publicacao_s:%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?&q=RENDIMENTOS+TRIBUT%C3%81VEIS+e+ALUGU%C3%89IS&fq=ano_publicacao_s:%222019%22"
    ],
    "21 RENDIMENTOS TRIBUTÁVEIS — PENSÃO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+TRIBUT%C3%81VEIS+e+PENS%C3%83O&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+TRIBUT%C3%81VEIS+e+PENS%C3%83O&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+TRIBUT%C3%81VEIS+e+PENS%C3%83O&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+TRIBUT%C3%81VEIS+e+PENS%C3%83O&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+TRIBUT%C3%81VEIS+e+PENS%C3%83O&fq=ano_publicacao_s%3A%222019%22"
    ],
    "23 RENDIMENTOS RECEBIDOS ACUMULADAMENTE (RRA)": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+RECEBIDOS+ACUMULADAMENTE&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+RECEBIDOS+ACUMULADAMENTE&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+RECEBIDOS+ACUMULADAMENTE&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+RECEBIDOS+ACUMULADAMENTE&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+RECEBIDOS+ACUMULADAMENTE&fq=ano_publicacao_s%3A%222019%22"
    ],
    "24 PESSOA FÍSICA EQUIPARADA A PESSOA JURÍDICA": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+PESSOA+F%C3%8DSICA+EQUIPARADA+A+PESSOA+JUR%C3%8DDICA&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+PESSOA+F%C3%8DSICA+EQUIPARADA+A+PESSOA+JUR%C3%8DDICA&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+PESSOA+F%C3%8DSICA+EQUIPARADA+A+PESSOA+JUR%C3%8DDICA&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+PESSOA+F%C3%8DSICA+EQUIPARADA+A+PESSOA+JUR%C3%8DDICA&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+PESSOA+F%C3%8DSICA+EQUIPARADA+A+PESSOA+JUR%C3%8DDICA&fq=ano_publicacao_s%3A%222019%22"
    ],
    "25 CARNÊ-LEÃO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+CARN%C3%8A-LE%C3%83O+&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+CARN%C3%8A-LE%C3%83O+&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+CARN%C3%8A-LE%C3%83O+&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+CARN%C3%8A-LE%C3%83O+&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+CARN%C3%8A-LE%C3%83O+&fq=ano_publicacao_s%3A%222019%22"
    ],
    "26 RENDIMENTOS ISENTOS E NÃO TRIBUTÁVEIS": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+ISENTOS+E+N%C3%83O+TRIBUT%C3%81VEIS&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+ISENTOS+E+N%C3%83O+TRIBUT%C3%81VEIS&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+ISENTOS+E+N%C3%83O+TRIBUT%C3%81VEIS&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+ISENTOS+E+N%C3%83O+TRIBUT%C3%81VEIS&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+RENDIMENTOS+ISENTOS+E+N%C3%83O+TRIBUT%C3%81VEIS&fq=ano_publicacao_s%3A%222019%22"
    ],
    "27 IMPOSTO SOBRE A RENDA INCIDENTE NA FONTE": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE+NA+FONTE&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE+NA+FONTE&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE+NA+FONTE&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE+NA+FONTE&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE+NA+FONTE&fq=ano_publicacao_s%3A%222019%22"
    ],
    "28 IMPOSTO SOBRE A RENDA INCIDENTE/RETIDO NA FONTE - TRIBUTAÇÃO EXCLUSIVA NA FONTE": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE%2FRETIDO+NA+FONTE+-+TRIBUTA%C3%87%C3%83O+EXCLUSIVA+NA+FONTE&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE%2FRETIDO+NA+FONTE+-+TRIBUTA%C3%87%C3%83O+EXCLUSIVA+NA+FONTE&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE%2FRETIDO+NA+FONTE+-+TRIBUTA%C3%87%C3%83O+EXCLUSIVA+NA+FONTE&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE%2FRETIDO+NA+FONTE+-+TRIBUTA%C3%87%C3%83O+EXCLUSIVA+NA+FONTE&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE%2FRETIDO+NA+FONTE+-+TRIBUTA%C3%87%C3%83O+EXCLUSIVA+NA+FONTE&fq=ano_publicacao_s%3A%222019%22"
    ],
    "29 IMPOSTO SOBRE A RENDA INCIDENTE/RETIDO NA FONTE - RESPONSABILIDADE DA FONTE PAGADORA": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE%2FRETIDO+NA+FONTE+-+RESPONSABILIDADE+DA+FONTE+PAGADORA&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE%2FRETIDO+NA+FONTE+-+RESPONSABILIDADE+DA+FONTE+PAGADORA&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE%2FRETIDO+NA+FONTE+-+RESPONSABILIDADE+DA+FONTE+PAGADORA&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE%2FRETIDO+NA+FONTE+-+RESPONSABILIDADE+DA+FONTE+PAGADORA&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE%2FRETIDO+NA+FONTE+-+RESPONSABILIDADE+DA+FONTE+PAGADORA&fq=ano_publicacao_s%3A%222019%22"
    ],
    "30 IMPOSTO SOBRE A RENDA INCIDENTE/RETIDO NA FONTE - FALTA DE RETENÇÃO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE%2FRETIDO+NA+FONTE+e+FALTA+DE+RETEN%C3%87%C3%83O&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE%2FRETIDO+NA+FONTE+e+FALTA+DE+RETEN%C3%87%C3%83O&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE%2FRETIDO+NA+FONTE+e+FALTA+DE+RETEN%C3%87%C3%83O&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE%2FRETIDO+NA+FONTE+e+FALTA+DE+RETEN%C3%87%C3%83O&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+IMPOSTO+SOBRE+A+RENDA+INCIDENTE%2FRETIDO+NA+FONTE+e+FALTA+DE+RETEN%C3%87%C3%83O&fq=ano_publicacao_s%3A%222019%22"
    ],
    "32 DEDUÇÕES – PREVIDÊNCIA": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+PREVID%C3%8ANCIA&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+PREVID%C3%8ANCIA&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+PREVID%C3%8ANCIA&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+PREVID%C3%8ANCIA&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+PREVID%C3%8ANCIA&fq=ano_publicacao_s%3A%222019%22"
    ],
    "33 DEDUÇÕES – DEPENDENTES": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+DEPENDENTES&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+DEPENDENTES&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+DEPENDENTES&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+DEPENDENTES&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+DEPENDENTES&fq=ano_publicacao_s%3A%222019%22"
    ],
    "34 DEDUÇÕES - PENSÃO ALIMENTÍCIA": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+PENS%C3%83O+ALIMENT%C3%8DCIA&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+PENS%C3%83O+ALIMENT%C3%8DCIA&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+PENS%C3%83O+ALIMENT%C3%8DCIA&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+PENS%C3%83O+ALIMENT%C3%8DCIA&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+PENS%C3%83O+ALIMENT%C3%8DCIA&fq=ano_publicacao_s%3A%222019%22"
    ],
    "35 DEDUÇÕES - DESPESAS MÉDICAS": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+DESPESAS+M%C3%89DICAS&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+DESPESAS+M%C3%89DICAS&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+DESPESAS+M%C3%89DICAS&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+DESPESAS+M%C3%89DICAS&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+DESPESAS+M%C3%89DICAS&fq=ano_publicacao_s%3A%222019%22"
    ],
    "36 DEDUÇÕES - DESPESAS COM INSTRUÇÃO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+DESPESAS+COM+INSTRU%C3%87%C3%83O&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+DESPESAS+COM+INSTRU%C3%87%C3%83O&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+DESPESAS+COM+INSTRU%C3%87%C3%83O&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+DESPESAS+COM+INSTRU%C3%87%C3%83O&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+DESPESAS+COM+INSTRU%C3%87%C3%83O&fq=ano_publicacao_s%3A%222019%22"
    ],
    "37 DEDUÇÕES - LIVRO-CAIXA": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+LIVRO-CAIXA&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+LIVRO-CAIXA&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+LIVRO-CAIXA&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+LIVRO-CAIXA&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+LIVRO-CAIXA&fq=ano_publicacao_s%3A%222019%22"
    ],
    "38 DEDUÇÕES - RENDIMENTOS DE ALUGUÉIS": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+RENDIMENTOS+DE+ALUGU%C3%89IS&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+RENDIMENTOS+DE+ALUGU%C3%89IS&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+RENDIMENTOS+DE+ALUGU%C3%89IS&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+RENDIMENTOS+DE+ALUGU%C3%89IS&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+DEDU%C3%87%C3%95ES+e+RENDIMENTOS+DE+ALUGU%C3%89IS&fq=ano_publicacao_s%3A%222019%22"
    ],
    "42 ATIVIDADE RURAL - CONCEITOS GERAIS": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ATIVIDADE+RURAL&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ATIVIDADE+RURAL&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ATIVIDADE+RURAL&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ATIVIDADE+RURAL&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ATIVIDADE+RURAL&fq=ano_publicacao_s%3A%222019%22"
    ],
    "45 ATIVIDADE RURAL - INVESTIMENTOS": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ATIVIDADE+RURAL+E+INVESTIMENTOS&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ATIVIDADE+RURAL+E+INVESTIMENTOS&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ATIVIDADE+RURAL+E+INVESTIMENTOS&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ATIVIDADE+RURAL+E+INVESTIMENTOS&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ATIVIDADE+RURAL+E+INVESTIMENTOS&fq=ano_publicacao_s%3A%222019%22"
    ],
    "46 GANHO DE CAPITAL": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+GANHO+DE+CAPITAL&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+GANHO+DE+CAPITAL&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+GANHO+DE+CAPITAL&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+GANHO+DE+CAPITAL&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+GANHO+DE+CAPITAL&fq=ano_publicacao_s%3A%222019%22"
    ],
    "47 ALIENAÇÃO DE BENS E DIREITOS": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ALIENA%C3%87%C3%83O+DE+BENS+E+DIREITOS&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ALIENA%C3%87%C3%83O+DE+BENS+E+DIREITOS&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ALIENA%C3%87%C3%83O+DE+BENS+E+DIREITOS&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ALIENA%C3%87%C3%83O+DE+BENS+E+DIREITOS&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+ALIENA%C3%87%C3%83O+DE+BENS+E+DIREITOS&fq=ano_publicacao_s%3A%222019%22"
    ],
    "48 BENS E DIREITOS - DISCRIMINAÇÃO NA DECLARAÇÃO": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+BENS+E+DIREITOS+DISCRIMINA%C3%87%C3%83O+NA+DECLARA%C3%87%C3%83O&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+BENS+E+DIREITOS+DISCRIMINA%C3%87%C3%83O+NA+DECLARA%C3%87%C3%83O&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+BENS+E+DIREITOS+DISCRIMINA%C3%87%C3%83O+NA+DECLARA%C3%87%C3%83O&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+BENS+E+DIREITOS+DISCRIMINA%C3%87%C3%83O+NA+DECLARA%C3%87%C3%83O&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+BENS+E+DIREITOS+DISCRIMINA%C3%87%C3%83O+NA+DECLARA%C3%87%C3%83O&fq=ano_publicacao_s%3A%222019%22"
    ],
    "49 BENS E DIREITOS - ÔNUS E DIREITOS REAIS": [
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+BENS+E+DIREITOS+ONUS+E+DIREITOS+REAIS&fq=ano_publicacao_s%3A%222023%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+BENS+E+DIREITOS+ONUS+E+DIREITOS+REAIS&fq=ano_publicacao_s%3A%222022%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+BENS+E+DIREITOS+ONUS+E+DIREITOS+REAIS&fq=ano_publicacao_s%3A%222021%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+BENS+E+DIREITOS+ONUS+E+DIREITOS+REAIS&fq=ano_publicacao_s%3A%222020%22",
        "https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+BENS+E+DIREITOS+ONUS+E+DIREITOS+REAIS&fq=ano_publicacao_s%3A%222019%22"
    ]

})


In [16]:
# Loop para percorrer todos os grupos
for grupo, links in links_por_grupo.items():
    grupo_path = os.path.join(base_path, grupo)
    criar_pasta(grupo_path)

    for link in links:
        # Capturar o ano antes de %22
        match = re.search(r"ano_publicacao_s%3A%22(\d{4})%22", link)
        if match:
            ano = match.group(1)
            print(f"Ano encontrado: {ano} no link: {link}")
        else:
            ano = "desconhecido"
            print(f"Padrão de ano não encontrado no link: {link}")

        ano_path = os.path.join(grupo_path, ano)
        criar_pasta(ano_path)

        current_url = link
        while current_url:
            pdf_links, next_url = scrape_page(current_url)
            for pdf_url in pdf_links:
                nome_pdf = pdf_url.split('/')[-1]
                caminho_pdf = os.path.join(ano_path, nome_pdf)
                download_pdf(pdf_url, caminho_pdf)
            current_url = next_url

print(f"Todos os acórdãos foram baixados e organizados em: {base_path}")

Ano encontrado: 2023 no link: https://acordaos.economia.gov.br/solr/acordaos2_shard10_replica_n56/browse?q=IRPF+e+OBRIGATORIEDADE+DE+APRESENTA%C3%87%C3%83O&fq=ano_publicacao_s%3A%222023%22
PDF baixado: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/1 OBRIGATORIEDADE DE APRESENTAÇÃO/2023/10580720800200967_6906872.pdf
PDF baixado: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/1 OBRIGATORIEDADE DE APRESENTAÇÃO/2023/12448720882201033_6914753.pdf
PDF baixado: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/1 OBRIGATORIEDADE DE APRESENTAÇÃO/2023/10580725415201040_6840501.pdf
PDF baixado: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/1 OBRIGATORIEDADE DE APRESENTAÇÃO/2023/18470723722201126_6974291.pdf
PDF baixado: /content/drive/Shareddrives/RAG Receita/Scraping Acórdãos/Acórdãos/1 OBRIGATORIEDADE DE APRESENTAÇÃO/2023/13896003929200861_6959144.pdf
PDF baixado: /content/drive/Shareddrives/RAG R

KeyboardInterrupt: 